## Predicting Apartment Prices in Mexico City 

Goal: A consolidation of the contents in 0-price-and size, 1-price-and-location, 2-price-and-neighborhood, and price-and-everything using the mexico dataset.

In [1]:
# Import libraries here
from glob import glob

import pandas as pd
import numpy as np
import seaborn as sns
from category_encoders import OneHotEncoder
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact


from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

### Prepare Data

#### Import

In [2]:
def wrangle(filepath):
    df = pd.read_csv(filepath)

    return df